# dsp4bats - user guide - 02 - get_files

This notebook is a part of the CloudedBats software project (http://cloudedbats.org).

Source code for dsp4bats: https://github.com/cloudedbats/cloudedbats_dsp

### Get a list of sound files and availabe metadata 

Sound files collected by using the CloudedBats WURB contains some useful information in
the filename. By using the SoundFileManager class it is possible to easily extract this
metadata and use it in a Pandas DataFrame.

The example code below shows how to find all wave files in a directory and to use the 
metadata to plot the recording positions on a map.

In [1]:
# Add path from jupyter to the dsp4bats library.
import sys
path = ".."
sys.path.append(path)

In [2]:
# Import python modules.
%matplotlib inline
import pandas as pd
import numpy as np
import dsp4bats

In [3]:
# tkinter is used for open file/dir dialogs.
import tkinter
import tkinter.filedialog
root = tkinter.Tk()
root.withdraw()
#file_path = filedialog.askopenfilename()
#dir_path = tkinter.filedialog.askdirectory()

''

In [4]:
# Create a file manager object.
file_mgr = dsp4bats.WurbFileUtils()

In [5]:
# Use Tkinter to select a directory and get all wave files found.
dir_path = tkinter.filedialog.askdirectory()
file_mgr.find_sound_files(dir_path=dir_path, recursive=True)

In [6]:
# Get and check the dataframe content.
files_df = file_mgr.get_dataframe()
files_df

,detector_id,datetime,datetime_str,latitude_dd,longitude_dd,latlong_str,rec_type,frame_rate_hz,file_frame_rate_hz,is_te,comments,dir_path,file_name,file_path,file_stem,abs_file_path
0,WurbAA03,2017-08-05 23:18:07+02:00,20170805T231807+0200,43.2619,-1.3506,N43.2619W1.3506,TE384,384000,38400,True,,/home/arnold/Desktop/bats_bidarrai,WurbAA03_20170805T231807+0200_N43.2619W1.3506_...,/home/arnold/Desktop/bats_bidarrai/WurbAA03_20...,WurbAA03_20170805T231807+0200_N43.2619W1.3506_...,/home/arnold/Desktop/bats_bidarrai/WurbAA03_20...


In [7]:
# Group by positions and count files at each position.
distinct_df = pd.DataFrame({'file_count' : files_df.groupby( [ 'latlong_str', 'latitude_dd', 'longitude_dd'] ).size()}).reset_index()

In [8]:
# Add a column for description to be shown when hovering over point in map.
distinct_df['description'] = 'Pos: ' + distinct_df['latlong_str'] + ' Count: ' + distinct_df['file_count'].astype(str)
distinct_df

,latlong_str,latitude_dd,longitude_dd,file_count,description
0,N43.2619W1.3506,43.2619,-1.3506,1,Pos: N43.2619W1.3506 Count: 1


In [9]:
# Plot on a map. There are many alternatives to do this
# but folium was easy to use for this example.
# Folium install: sudo pip3 install folium
import folium

# Use the mean value as center for the map.
center_lat = distinct_df.latitude_dd.mean()
center_long = distinct_df.longitude_dd.mean()
# Create map object.
map_osm = folium.Map(location=[center_lat, center_long], zoom_start=10)
# Loop over positions an create markers.
for long, lat, desc in zip(distinct_df.longitude_dd.values,
                         distinct_df.latitude_dd.values,
                         distinct_df.description.values):
    # The description column is used for popup messages.
    marker = folium.Marker([lat, long], popup=desc).add_to(map_osm)

# Display in notebook.
#map_osm

# Write to html file.
map_osm.save('data_out/map_folium.html')